In [ ]:
import numpy as np
import pandas as pd
from xgboost import XGBRegressor
import matplotlib.pyplot as plt


In [ ]:
# Load trained model
model = XGBRegressor()
model.load_model("xgboost_demand_model.json")

# Load residuals
residuals = pd.read_csv("demand_residuals_xgboost_morefeatures.csv")["Residual"].values

# Load last known feature row
last_features = pd.read_csv("last_known_features_xgboost_morefeatures.csv")

# Feature column order
feature_cols = pd.read_csv("feature_columns_xgboost_morefeatures.csv")["feature_name"].tolist()
last_features = last_features[feature_cols]


In [ ]:
N_SIMULATIONS = 5000
HORIZON_DAYS = 60

INITIAL_INVENTORY = 600
REORDER_POINT = 400
ORDER_QUANTITY = 500
LEAD_TIME_DAYS = 14


In [ ]:
def run_single_simulation():
    inventory = INITIAL_INVENTORY
    pending_orders = []
    inventory_history = []
    stockout = False

    features = last_features.copy()

    for day in range(HORIZON_DAYS):

        # Predict mean demand
        mean_demand = model.predict(features)[0]

        # Add uncertainty
        demand = max(0, mean_demand + np.random.choice(residuals))

        # Receive orders
        pending_orders = [(d-1, q) for d, q in pending_orders]
        arrivals = [q for d, q in pending_orders if d <= 0]
        inventory += sum(arrivals)
        pending_orders = [(d, q) for d, q in pending_orders if d > 0]

        # Apply demand
        if demand > inventory:
            stockout = True
            inventory = 0
        else:
            inventory -= demand

        inventory_history.append(inventory)

        # Reorder logic
        if inventory <= REORDER_POINT:
            pending_orders.append((LEAD_TIME_DAYS, ORDER_QUANTITY))

        # Update lag features (rolling forward)
        features["lag_30"] = features["lag_14"]
        features["lag_14"] = features["lag_7"]
        features["lag_7"] = features["lag_1"]
        features["lag_1"] = demand

        features["roll_mean_7"] = (
            features["roll_mean_7"] * 6 + demand
        ) / 7

    return stockout, inventory_history


In [ ]:
stockouts = 0
all_inventory_paths = []

for _ in range(N_SIMULATIONS):
    so, inv_hist = run_single_simulation()
    stockouts += int(so)
    all_inventory_paths.append(inv_hist)

all_inventory_paths = np.array(all_inventory_paths)


In [ ]:
stockout_probability = stockouts / N_SIMULATIONS
avg_inventory = all_inventory_paths.mean()

print("Monte Carlo Inventory Simulation Results")
print("---------------------------------------")
print(f"Stockout Probability : {stockout_probability:.4f}")
print(f"Average Inventory   : {avg_inventory:.2f}")


In [ ]:
plt.figure(figsize=(10,4))

for i in range(50):
    plt.plot(all_inventory_paths[i], alpha=0.3)

plt.axhline(REORDER_POINT, linestyle="--", color="red", label="Reorder Point")
plt.xlabel("Day")
plt.ylabel("Inventory Level")
plt.title("Sample Inventory Trajectories (Monte Carlo)")
plt.legend()
plt.tight_layout()
plt.show()
